# Tutorial 2: Influence of magnetic field on FM and AFM skyrmions size 

In this tutorial, we will investigate the effect of an external magnetic field pointing in the z direction on the radius of both FM skyrmion and AFM skyrmion on a square lattice.

The Heisenberg Hamiltonian includes an additional term, which is the Zeeman term that is colored in red:

$$
H = - \sum_{\langle i, j \rangle} J_{ij} \mathbf{S}_i \cdot \mathbf{S}_j  
-  \sum_{\langle i,j \rangle} D_{ij} \cdot  (\mathbf{S}_i \times \mathbf{S}_j)
- \sum_i K_i (S_i^z)^2 
- \textcolor{red}{\sum_i \mathbf{B} \cdot \mathbf{S}_i}
$$

The last term represents the Zeeman interaction.



## FM skyrmion

The starting point is the FM skyrmion shown below, where the spins are saved in **FM_skyrmion.ovf** spin file.
The magnetic interactions among the atoms are as shown below, where the DMI vector components stabilize the Bloch skyrmion as shown in Tutorial one. 


<p align="center">
    <img src="./assets/bloch.png" alt="Bloch skyrmion" style="display: block; margin-left: auto; margin-right: auto; width:20%;">
</p>

The magnetic interactions among the atoms are as shown below: 

| Interaction | Value (meV) |
|------------|-------------|
| J        | 4           |
| D        | 0.4         |
| K         | 0.12       |


The interactions are read in the input file from an external file **Square_FM.txt**

In [ ]:
!sed -n '48,49p' input_FM_skyrmion.cfg

By running the Python script **"Magnetic_field_FM_skyrmion.py"**, the script performs the following steps:

1. Reads the input file **"input_FM_skyrmion.cfg"**.
2. Loads the initial magnetic state from **"FM_skyrmion.ovf"**.
3. Applies an out-of-plane (OOP) magnetic field, varying from **0.0 to 2.4 meV** in steps of **0.2 meV**.
4. Starts the Atomistic Spin Dynamics (ASD) simulation to minimize the system's energy.
5. Writes the converged spin configurations as **"spins_B{B:.2f}.ovf"**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from spirit import state, configuration, simulation, geometry, system, hamiltonian, io

import Magnetic_field_FM_skyrmion as fm

# Simulation setup
cfgfile = "input_FM_skyrmion.cfg"
quiet = True
B_values = np.arange(0.00, 2.4, 0.2)
radii = []

for B in B_values:
    with state.State(cfgfile, quiet) as p_state:
        io.image_read(p_state, "FM_skyrmion.ovf")
        hamiltonian.set_field(p_state, B, [0, 0, 1], idx_image=-1, idx_chain=-1)
        simulation.start(p_state, simulation.METHOD_LLG, simulation.SOLVER_LBFGS_OSO)
        system.update_data(p_state)
        spin_file = f"spins_B{B:.2f}.ovf"
        io.image_write(p_state, spin_file)
        positions = geometry.get_positions(p_state)
        spins = np.loadtxt(spin_file)

        profile, radius, popt = fm.get_profile(positions, spins, f"B{B:.2f}")
        radii.append(radius)


In [ ]:
def plot_radii(B_values, radii, label):
    """plot non-zero skrmion radii"""
    B_values = np.asarray(B_values)
    radii = np.asarray(radii)

    fig, ax = plt.subplots(1, 1, figsize=(10, 6))

    # Filter out zero radius values before plotting
    mask = radii > 0

    # Plot radius vs B value only for nonzero radii
    ax.plot(B_values[mask], radii[mask], marker='o')
    ax.set_title(f'{label} skyrmion radius vs magnetic field strength')
    ax.set_xlabel('B (meV)')
    ax.set_ylabel('Radius (Å)')
    ax.grid()
    fig.savefig(f'{label}_radius_vs_B.png')

    return fig, ax


In [ ]:
fig, ax = plot_radii(B_values, radii, "FM")


6. Plots the resulting magnetic state at each magnetic field value, indicating the skyrmion radius if it still exists as shown in the figure below.

<p align="center">
    <img src="./assets/Bloch-example.png" alt="Bloch skyrmion" style="display: block; margin-left: auto; margin-right: auto; width:30%;">
</p>


   
7. Generates a final plot of **magnetic field strength vs. skyrmion radius**.

<p align="center">
    <img src="./assets/raduis.png" alt="Bloch skyrmion" style="display: block; margin-left: auto; margin-right: auto; width:30%;">
</p>


# AFM skyrmion:
In this part, we do the same as has been done in the first part, but here we apply the magnetic field on an AFM skyrmion.
The interactions for this example are as shown below, which are needed to satbilize a Néel  AFM skyrmion on a square lattice.
The interactions are saved in **Square_AFM.txt** file  


 
| Interaction | Value (meV) |
|------------|-------------|
| J1        | -6           |
| J2        | 2          |
| D1        | 0.02         |
| D2        | 0.08           |
| K         | 0.006        |


Those interactions stabilize a Néel skyrmion on a square lattice as shown below: 


<p align="center">
    <img src="./assets/AFM_skyrmion.png" alt="AFM skyrmion" style="display: block; margin-left: auto; margin-right: auto; width:30%;">
</p>


By executing the Python script **Magnetic_field_AFM_skyrmion.py**, you can observe the impact of the out-of-plane (OOP) magnetic field on the radius of AFM skyrmions by examining the plots corresponding to different magnetic field strengths.

<p align="center">
    <img src="./assets/afm_sky-B.png" alt="AFM skyrmion" style="display: block; margin-left: auto; margin-right: auto; width:30%;">
</p>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from spirit import state, configuration, simulation, geometry, system, hamiltonian, io

import Magnetic_field_AFM_skyrmion as afm

# Simulation setup
cfgfile = "input_AFM_skyrmion.cfg"
quiet = True

B_values = np.arange(0.00, 30, 5)  # Changing applied B from 0 to 30 at step 5
radii = []

for B in B_values:
    with state.State(cfgfile, quiet) as p_state:
        io.image_read(p_state, "AFM_skyrmion.ovf")
        hamiltonian.set_field(p_state, B, [0, 0, 1], idx_image=-1, idx_chain=-1)
        simulation.start(p_state, simulation.METHOD_LLG, simulation.SOLVER_LBFGS_OSO)
        system.update_data(p_state)

        # Get original spins and positions
        spins = system.get_spin_directions(p_state)
        positions = geometry.get_positions(p_state)

        # Initialize new lists
        spins_new = []
        positions_new = []

        # Get lattice structure
        n_cells = geometry.get_n_cells(p_state)     # number of cells in Bravais lattice
        n_cell_atoms = geometry.get_n_cell_atoms(p_state)  # number of basis atoms per unit cell

        # Filter spins and positions
        for c in range(n_cells[2]):
            for b in range(n_cells[1]):
                for a in range(n_cells[0]):
                    for i in range(n_cell_atoms):
                        sublattice = (a % 2)
                        sublattice_2 = (b % 2)
                        if sublattice_2 == sublattice:
                            idx = i + n_cell_atoms * (a + n_cells[0] * (b + n_cells[1] * c))
                            spins_new.append(spins[idx])
                            positions_new.append(positions[idx])

        # Convert to numpy arrays
        spins_new = np.array(spins_new)
        positions_new = np.array(positions_new)

        # Save to files (optional, for debugging)
        np.savetxt(f"spins_B{B:.2f}.txt", spins_new)
        # np.savetxt(f"positions_B{B:.2f}.txt", positions_new)

        # Compute profile and radius using filtered spins/positions
        profile, radius, popt = afm.get_profile(positions_new, spins_new, positions, spins, f"B{B:.2f}")

        # Store radius value
        radii.append(radius)


In [ ]:
fig, ax = plot_radii(B_values, radii, "AFM")
